# load data

In [1]:
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse

In [2]:
from datetime import date
from datetime import timedelta

## load tweets older then two weeks

In [3]:
two_weeks_old = date.today() - timedelta(days=14)

In [4]:
cnx = sqlite3.connect("../home.db")
query = f"SELECT id,user, full_text, created_at, lang FROM tweets WHERE created_at < '{str(two_weeks_old)}'"
print(query)
df_tweets = pd.read_sql_query(
    query,
    cnx,
)

SELECT id,user, full_text, created_at, lang FROM tweets WHERE created_at < '2021-01-07'


In [5]:
df_tweets.tail()

,id,user,full_text,created_at,lang
39753,1346950097033179137,851958139851296771,The man pictured in Nancy Pelosi’s office took...,2021-01-06T22:41:42+00:00,en
39754,1346959019907416064,381289719,"If someone, a year ago, described January 6, 2...",2021-01-06T23:17:10+00:00,en
39755,1346959100794474498,197176970,Holocaust denier Nick Fuentes and neo-nazi los...,2021-01-06T23:17:29+00:00,en
39756,1346959937679798274,13666,WhatsApp will share its users' personal inform...,2021-01-06T23:20:49+00:00,en
39757,1346963410966048768,45122898,You know the word we're not hearing enough rig...,2021-01-06T23:34:37+00:00,en


# utils

In [6]:
def find_url(tweet):
    return re.findall("http\S+", tweet)


def clean_links(tweet):
    tweet = re.sub(r"bit.ly/\S+", "", tweet)
    tweet = re.sub(r"t.co/\S+", "", tweet)
    tweet = re.sub(r"buff.ly/\S+", "", tweet)
    tweet = re.sub(r"twitter.com/\S+", "", tweet)
    return tweet


def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain


def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return l

# function for df

In [7]:
def find_news(df, news_domains_list):

    df["urls"] = df["full_text"].apply(find_url)
    df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

In [8]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [9]:
%%time
df_tweets = find_news(df_tweets, news_domains)

CPU times: user 461 ms, sys: 4.22 ms, total: 465 ms
Wall time: 469 ms


In [10]:
df_tweets.head()

,id,user,full_text,created_at,lang,contains_news
0,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,0
1,70261648811761665,5416652,"I wish I had kept my 1,700 BTC @ $0.06 instead...",2011-05-16T22:57:37+00:00,en,0
2,177008089394970624,5110861,"In 1996 a man took a NZ radio station hostage,...",2012-03-06T12:29:51+00:00,en,1
3,234002950274560000,108471631,What is she thinking? https://twitter.com/MELA...,2012-08-10T19:07:06+00:00,en,0
4,281811460718477312,16298441,did you know that the bible doesn't actually c...,2012-12-20T17:21:02+00:00,en,0


# remove seen

In [11]:
seen_tweets = pd.read_csv("seen.csv")

In [12]:
seen_tweets.head()

,tweet_id,err_reason
0,1337875267096875015,no_errors
1,1335698027537969155,no_errors
2,1345699105385959435,not_found
3,1336874318882467842,no_errors
4,1337717825629253633,not_found


In [13]:
seen_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_id    2200 non-null   int64 
 1   err_reason  2200 non-null   object
dtypes: int64(1), object(1)
memory usage: 34.5+ KB


In [14]:
seen_tweets.drop_duplicates(inplace=True)

In [15]:
df_tweets[df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())].shape

(2200, 6)

In [16]:
%%time
print(df_tweets.shape)
df_tweets = df_tweets[~df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())]

(39758, 6)
CPU times: user 6.21 ms, sys: 129 µs, total: 6.34 ms
Wall time: 5.8 ms


In [17]:
# fiter out non english tweets

In [18]:
df_tweets.shape

(37558, 6)

In [19]:
df_tweets = df_tweets[df_tweets["lang"] == "en"]
df_tweets.shape

(34469, 6)

# suffle them and filter out news

In [20]:
df_tweets[df_tweets["contains_news"] == 0].shape

(32823, 6)

In [21]:
to_custom_news_feed = (
    df_tweets[df_tweets["contains_news"] == 0]
    .sample(frac=1)
    .reset_index(drop=True)[:1000]
)
to_custom_news_feed.shape

(1000, 6)

In [22]:
to_custom_news_feed[["id", "user"]].to_csv("batch_to_add.csv")